In [1]:
import os
import sys

import pandas as pd
import xarray as xr

import fine as fn
from fine.aggregations.technologyAggregation.techAggregation import (
    aggregate_RE_technology,
)
import fine.IOManagement.xarrayIO as xrIO

cwd = os.getcwd()


# Workflow for technology aggregation 

This example notebook shows how Variable Renewable Energy Sources (VRES) can be aggregated to obtain fewer types.

<img src="figures/tech_aggregation_for_example_notebook.png" style="width: 1000px;"/>

The figure above depicts the basic idea behind technology aggregation. Here, the number of VRES within each region is reduced to a desired number. To give you an example, if the results of your PV simulation are spatially detailed or spatially highly resolved, then you could reduce these to a few types within each region. The time series profiles are matched during grouping of these technologies. 


# STEP 1. Technology Aggregation 

In [2]:
# The input data for technology aggregation could either be provided in gridded or non-gridded form.
## The example here shows the data in the gridded form. In this case, a corresponding shapefile should also be provided to
## match the data in the grids to appropriate regions. Please refer to the documentation to know more.

# shapefile containing model regions' geometries
SHP_PATH = os.path.join(cwd, "output_data", "aggregated_regions.shp")

# netcdf files containing highly resolved VRES data. In this example, both PV and wind turbines are aggregated

ONSHORE_WIND_DATA_PATH = os.path.join(
    cwd, "input_tech_aggregation_data", "DEU_wind.nc4"
)
PV_DATA_PATH = os.path.join(cwd, "input_tech_aggregation_data", "DEU_pv.nc4")

In [3]:
# Let us first take a look at one of these datasets

xr.open_dataset(ONSHORE_WIND_DATA_PATH)

<xarray.Dataset> Size: 361MB
Dimensions:      (y: 117, x: 88, time: 8760)
Coordinates:
  * y            (y) float64 936B 2.832e+06 2.838e+06 ... 3.408e+06 3.412e+06
  * x            (x) float64 704B 4.092e+06 4.098e+06 ... 4.522e+06 4.528e+06
    spatial_ref  int32 4B ...
  * time         (time) datetime64[ns] 70kB 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    capacity     (y, x) float64 82kB ...
    capfac       (time, y, x) float32 361MB ...
Attributes:
    xy_reference_system:  EPSG:3035
    grid_mapping:         spatial_ref

In [4]:
## Aggregation
aggregated_wind_ds = aggregate_RE_technology(
    ONSHORE_WIND_DATA_PATH,
    "xy_reference_system",
    SHP_PATH,
    n_timeSeries_perRegion=5,
    capacity_var_name="capacity",
    capfac_var_name="capfac",
    shp_index_col="space",
)

aggregated_pv_ds = aggregate_RE_technology(
    PV_DATA_PATH,
    "xy_reference_system",
    SHP_PATH,
    n_timeSeries_perRegion=5,
    capacity_var_name="capacity",
    capfac_var_name="capfac",
    shp_index_col="space",
)

elapsed time for aggregate_RE_technology: 1.17 minutes
elapsed time for aggregate_RE_technology: 1.15 minutes


In [5]:
aggregated_wind_ds

<xarray.Dataset> Size: 1MB
Dimensions:     (region_ids: 4, TS_ids: 5, time: 8760)
Coordinates:
  * region_ids  (region_ids) object 32B 'cluster_0_cluster_2' ... 'cluster_7'
  * TS_ids      (TS_ids) <U4 80B 'TS_0' 'TS_1' 'TS_2' 'TS_3' 'TS_4'
  * time        (time) datetime64[ns] 70kB 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    capacity    (region_ids, TS_ids) float64 160B 6.236e+04 ... 2.296e+04
    capfac      (time, region_ids, TS_ids) float64 1MB 0.384 0.7 ... 0.2 0.7
Attributes: (12/20)
    cluster_0_cluster_2.TS_0:                      [(4132500.0, 3092500.0), (...
    cluster_0_cluster_2.TS_1:                      [(4147500.0, 2937500.0), (...
    cluster_0_cluster_2.TS_2:                      [(4092500.0, 3037500.0), (...
    cluster_0_cluster_2.TS_3:                      [(4217500.0, 3077500.0)]
    cluster_0_cluster_2.TS_4:                      [(4097500.0, 3087500.0), (...
    cluster_1_cluster_3_cluster_5_cluster_6.TS_0:  [(4162500.0, 3292500.0), (...
    ...                                            ...
    cluster_4.TS_4:                                [(4357500.0, 2942500.0)]
    cluster_7.TS_0:                                [(4092500.0, 2902500.0), (...
    cluster_7.TS_1:                                [(4102500.0, 2927500.0), (...
    cluster_7.TS_2:                                [(4127500.0, 2927500.0)]
    cluster_7.TS_3:                                [(4107500.0, 3002500.0)]
    cluster_7.TS_4:                                [(4107500.0, 2907500.0), (...

In [6]:
aggregated_pv_ds

<xarray.Dataset> Size: 1MB
Dimensions:     (region_ids: 4, TS_ids: 5, time: 8760)
Coordinates:
  * region_ids  (region_ids) object 32B 'cluster_0_cluster_2' ... 'cluster_7'
  * TS_ids      (TS_ids) <U4 80B 'TS_0' 'TS_1' 'TS_2' 'TS_3' 'TS_4'
  * time        (time) datetime64[ns] 70kB 2015-01-01 ... 2015-12-31T23:00:00
Data variables:
    capacity    (region_ids, TS_ids) float64 160B 4.03e+07 ... 1.746e+04
    capfac      (time, region_ids, TS_ids) float64 1MB 0.1992 0.1361 ... 0.2 0.3
Attributes: (12/20)
    cluster_0_cluster_2.TS_0:                      [(4092500.0, 3037500.0), (...
    cluster_0_cluster_2.TS_1:                      [(4167500.0, 2962500.0), (...
    cluster_0_cluster_2.TS_2:                      [(4267500.0, 2987500.0)]
    cluster_0_cluster_2.TS_3:                      [(4122500.0, 3082500.0)]
    cluster_0_cluster_2.TS_4:                      [(4252500.0, 2932500.0)]
    cluster_1_cluster_3_cluster_5_cluster_6.TS_0:  [(4162500.0, 3292500.0), (...
    ...                                            ...
    cluster_4.TS_4:                                [(4362500.0, 2932500.0), (...
    cluster_7.TS_0:                                [(4097500.0, 3012500.0), (...
    cluster_7.TS_1:                                [(4092500.0, 2902500.0), (...
    cluster_7.TS_2:                                [(4092500.0, 2927500.0)]
    cluster_7.TS_3:                                [(4097500.0, 2967500.0)]
    cluster_7.TS_4:                                [(4107500.0, 2937500.0), (...

# STEP 2. Adding the results to ESM instance 

In [7]:
# If you have an ESM instance, then you could add the results of technology aggregation to this instance

In [8]:
# First, set up the ESM instance
esm = xrIO.readNetCDFtoEnergySystemModel(
    os.path.join(cwd, "output_data", "aggregated_xr_ds.nc")
)

C:\Users\t.gross\Documents\Programming\Jugit\fine\fine\utils.py:1238: UserWarning: A declaration of bigM is not necessary if hasIsBuiltBinaryVariable is set to false. The value of bigM will be ignored in the optimization.
  warnings.warn(


In [9]:
# If wind turbine and PV are already present in the ESM instance, we need to replace them like shown in the cells below.
esm.componentModelingDict["SourceSinkModel"].componentsDict

{'Wind (onshore)': <fine.sourceSink.Source at 0x2072f3c1c90>,
 'PV': <fine.sourceSink.Source at 0x2072f323f40>,
 'Electricity demand': <fine.sourceSink.Sink at 0x2070128c8e0>,
 'Hydrogen demand': <fine.sourceSink.Sink at 0x2070128eef0>}

In [10]:
## First, get certain info corresponding to these techs as they remain the same:
var_list = ["processedInvestPerCapacity", "processedOpexPerCapacity", "interestRate", "economicLifetime"]

wind_vars = {}
pv_vars = {}

for var in var_list:
    wind_vars.update({var: esm.getComponentAttribute("Wind (onshore)", var).mean()})
    pv_vars.update({var: esm.getComponentAttribute("PV", var).mean()})

## And now we delete them
esm.removeComponent("Wind (onshore)")
esm.removeComponent("PV")

In [11]:
esm.componentModelingDict["SourceSinkModel"].componentsDict

{'Electricity demand': <fine.sourceSink.Sink at 0x2070128c8e0>,
 'Hydrogen demand': <fine.sourceSink.Sink at 0x2070128eef0>}

In [12]:
## Prepare the aggregation results and add them to the esm
data = {}

time_steps = esm.totalTimeSteps
regions = aggregated_wind_ds["region_ids"].values
clusters = aggregated_wind_ds["TS_ids"].values  # technology types per region


for i, cluster in enumerate(clusters):
    # Add a wind turbine
    data.update(
        {
            f"Wind (onshore), capacityMax {i}": pd.Series(
                aggregated_wind_ds.capacity.loc[:, cluster], index=regions
            )
        }
    )

    data.update(
        {
            f"Wind (onshore), operationRateMax {i}": pd.DataFrame(
                aggregated_wind_ds.capfac.loc[:, :, cluster].values,
                index=time_steps,
                columns=regions,
            )
        }
    )

    # Add a pv
    data.update(
        {
            f"PV, capacityMax {i}": pd.Series(
                aggregated_pv_ds.capacity.loc[:, cluster], index=regions
            )
        }
    )

    data.update(
        {
            f"PV, operationRateMax {i}": pd.DataFrame(
                aggregated_pv_ds.capfac.loc[:, :, cluster].values,
                index=time_steps,
                columns=regions,
            )
        }
    )

In [13]:
## add the data
for i, cluster in enumerate(clusters):
    esm.add(
        fn.Source(
            esM=esm,
            name=f"Wind (onshore) {i}",
            commodity="electricity",
            hasCapacityVariable=True,
            operationRateMax=data[f"Wind (onshore), operationRateMax {i}"],
            capacityMax=data[f"Wind (onshore), capacityMax {i}"],
            investPerCapacity=wind_vars.get("processedInvestPerCapacity"),
            opexPerCapacity=wind_vars.get("processedOpexPerCapacity"),
            interestRate=wind_vars.get("interestRate"),
            economicLifetime=wind_vars.get("economicLifetime"),
        )
    )

    esm.add(
        fn.Source(
            esM=esm,
            name=f"PV {i}",
            commodity="electricity",
            hasCapacityVariable=True,
            operationRateMax=data[f"PV, operationRateMax {i}"],
            capacityMax=data[f"PV, capacityMax {i}"],
            investPerCapacity=pv_vars.get("processedInvestPerCapacity"),
            opexPerCapacity=pv_vars.get("processedOpexPerCapacity"),
            interestRate=pv_vars.get("interestRate"),
            economicLifetime=pv_vars.get("economicLifetime"),
        )
    )

In [14]:
esm.componentModelingDict["SourceSinkModel"].componentsDict

{'Electricity demand': <fine.sourceSink.Sink at 0x2070128c8e0>,
 'Hydrogen demand': <fine.sourceSink.Sink at 0x2070128eef0>,
 'Wind (onshore) 0': <fine.sourceSink.Source at 0x2072f2fc4f0>,
 'PV 0': <fine.sourceSink.Source at 0x2072f323b80>,
 'Wind (onshore) 1': <fine.sourceSink.Source at 0x20700fc7790>,
 'PV 1': <fine.sourceSink.Source at 0x20700f6e4a0>,
 'Wind (onshore) 2': <fine.sourceSink.Source at 0x20700f6f5b0>,
 'PV 2': <fine.sourceSink.Source at 0x20700f762c0>,
 'Wind (onshore) 3': <fine.sourceSink.Source at 0x20700f77b80>,
 'PV 3': <fine.sourceSink.Source at 0x20700f820e0>,
 'Wind (onshore) 4': <fine.sourceSink.Source at 0x20700f83fd0>,
 'PV 4': <fine.sourceSink.Source at 0x20700f91f00>}

# Step 4. Temporal Aggregation 

In [15]:
esm.aggregateTemporally(numberOfTypicalPeriods=7)


Clustering time series data with 7 typical periods and 24 time steps per period 
further clustered to 12 segments per period...
		(8.8650 sec)



# Step 5. Optimization

In [16]:
esm.optimize(timeSeriesAggregation=True)
# The following `optimizationSpecs` are recommended if you use the Gurobi solver.
# aggregated_esM.optimize(timeSeriesAggregation=True,
#                         optimizationSpecs='OptimalityTol=1e-3 method=2 cuts=0')

Time series aggregation specifications:
Number of typical periods:7, number of time steps per period:24, number of segments per period:12

Declaring sets, variables and constraints for SourceSinkModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.2534 sec)

Declaring sets, variables and constraints for ConversionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0271 sec)

Declaring sets, variables and constraints for StorageModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.3176 sec)

Declaring sets, variables and constraints for LOPFModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0382 sec)

Declaring sets, variables and constraints for TransmissionModel
	declaring sets... 
	declaring variables... 
	declaring constraints... 
		(0.0381 sec)

Declaring shared potential constraint...
		(0.0000 sec)

Declaring linked component quantity constraint...
		(0.0000 s